# Face

In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
from tqdm import tqdm

# Input and output paths
input_dir = r"C:\Users\rames\OneDrive\Desktop\Bio 100\data"
output_dir = r"C:\Users\rames\OneDrive\Desktop\Bio 100"
modalities = ["face", "iris", "fingerprint"]
splits = [("Train", 70), ("Test", 15), ("Validation", 15)]

# Create output folder structure
for modality in modalities:
    for split, _ in splits:
        os.makedirs(os.path.join(output_dir, modality, split), exist_ok=True)

# Augmentation function
def augment_image(image):
    aug_images = []

    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    aug_images.append(pil_image)

    # Flips
    aug_images.append(pil_image.transpose(Image.FLIP_LEFT_RIGHT))
    aug_images.append(pil_image.transpose(Image.FLIP_TOP_BOTTOM))

    # Rotation
    for angle in [10, -10, 30, -30, 45, -45]:
        aug_images.append(pil_image.rotate(angle))

    # Sharpness
    enhancer = ImageEnhance.Sharpness(pil_image)
    aug_images.append(enhancer.enhance(2.0))

    # Blur
    blurred = cv2.GaussianBlur(np.array(pil_image), (5, 5), 0)
    aug_images.append(Image.fromarray(blurred))

    # Brightness
    enhancer = ImageEnhance.Brightness(pil_image)
    aug_images.append(enhancer.enhance(1.5))

    # Contrast
    enhancer = ImageEnhance.Contrast(pil_image)
    aug_images.append(enhancer.enhance(1.5))

    # Resize and trim to 100
    final_images = []
    for img in aug_images:
        img = img.resize((224, 224))
        final_images.append(img)
        if len(final_images) >= 100:
            break

    while len(final_images) < 100:
        final_images.append(random.choice(final_images))

    return final_images[:100]

# Main processing
for modality in modalities:
    csv_data = {"Train": [], "Test": [], "Validation": []}

    for person_folder in tqdm(os.listdir(input_dir), desc=f"Processing {modality}"):
        person_path = os.path.join(input_dir, person_folder)
        if not os.path.isdir(person_path):
            continue

        image_folder = os.path.join(person_path, modality)
        if not os.path.exists(image_folder):
            continue

        image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not image_files:
            continue

        image_path = os.path.join(image_folder, image_files[0])
        image = cv2.imread(image_path)
        if image is None:
            continue

        image = cv2.resize(image, (224, 224))
        augmented_images = augment_image(image)

        start = 0
        for split_name, count in splits:
            split_dir = os.path.join(output_dir, modality, split_name, person_folder)
            os.makedirs(split_dir, exist_ok=True)

            for i in range(count):
                img = augmented_images[start + i]
                filename = f"{person_folder}_{i+1}.jpg"
                save_path = os.path.join(split_dir, filename)
                img.convert("RGB").save(save_path, format="JPEG")

                relative_path = os.path.relpath(save_path, output_dir)
                csv_data[split_name].append([relative_path, person_folder])

            start += count

    # Save CSV files
    for split_name, _ in splits:
        csv_path = os.path.join(output_dir, modality, f"{split_name}.csv")
        df = pd.DataFrame(csv_data[split_name], columns=["image_path", "label"])
        df.to_csv(csv_path, index=False)

print("✅ All modalities processed and CSVs generated successfully!")


Processing fingerprint: 100%|██████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1773.62it/s]

✅ All modalities processed and CSVs generated successfully!


# Iris and Fingerprint

In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
from tqdm import tqdm

# Input and output paths
input_dir = r"C:\Users\rames\OneDrive\Desktop\Bio 100\data"
output_dir = r"C:\Users\rames\OneDrive\Desktop\Bio 100"
modalities = ["iris", "fingerprint"]  # excluding 'face'
splits = [("Train", 70), ("Test", 15), ("Validation", 15)]

# Valid image extensions
valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp')

# Create output folder structure
for modality in modalities:
    for split, _ in splits:
        os.makedirs(os.path.join(output_dir, modality, split), exist_ok=True)

# Augmentation function
def augment_image(image):
    aug_images = []
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    aug_images.append(pil_image)

    # Flips
    aug_images.append(pil_image.transpose(Image.FLIP_LEFT_RIGHT))
    aug_images.append(pil_image.transpose(Image.FLIP_TOP_BOTTOM))

    # Rotation
    for angle in [10, -10, 30, -30, 45, -45]:
        aug_images.append(pil_image.rotate(angle))

    # Sharpness
    enhancer = ImageEnhance.Sharpness(pil_image)
    aug_images.append(enhancer.enhance(2.0))

    # Blur
    blurred = cv2.GaussianBlur(np.array(pil_image), (5, 5), 0)
    aug_images.append(Image.fromarray(blurred))

    # Brightness
    enhancer = ImageEnhance.Brightness(pil_image)
    aug_images.append(enhancer.enhance(1.5))

    # Contrast
    enhancer = ImageEnhance.Contrast(pil_image)
    aug_images.append(enhancer.enhance(1.5))

    # Resize and trim to 100
    final_images = []
    for img in aug_images:
        img = img.resize((224, 224))
        final_images.append(img)
        if len(final_images) >= 100:
            break

    while len(final_images) < 100:
        final_images.append(random.choice(final_images))

    return final_images[:100]

# Main processing
for modality in modalities:
    csv_data = {"Train": [], "Test": [], "Validation": []}

    for person_folder in tqdm(os.listdir(input_dir), desc=f"Processing {modality}"):
        person_path = os.path.join(input_dir, person_folder)
        if not os.path.isdir(person_path):
            continue

        image_folder = os.path.join(person_path, modality)
        if not os.path.exists(image_folder):
            print(f"❌ Skipping {person_folder}: no {modality} folder")
            continue

        image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(valid_exts)]
        if not image_files:
            print(f"❌ Skipping {person_folder}/{modality}: no valid image files")
            continue

        image_path = os.path.join(image_folder, image_files[0])
        image = cv2.imread(image_path)
        if image is None:
            print(f"⚠️ Cannot read image: {image_path}")
            continue

        image = cv2.resize(image, (224, 224))
        augmented_images = augment_image(image)

        if len(augmented_images) < sum(count for _, count in splits):
            print(f"⚠️ Not enough augmented images for {person_folder} in {modality}, skipping.")
            continue

        start = 0
        for split_name, count in splits:
            split_dir = os.path.join(output_dir, modality, split_name, person_folder)
            os.makedirs(split_dir, exist_ok=True)

            for i in range(count):
                img = augmented_images[start + i]
                filename = f"{person_folder}_{i+1}.jpg"
                save_path = os.path.join(split_dir, filename)
                img.convert("RGB").save(save_path, format="JPEG")

                relative_path = os.path.relpath(save_path, output_dir)
                csv_data[split_name].append([relative_path, person_folder])

            start += count

    # Save CSV files
    for split_name, _ in splits:
        csv_path = os.path.join(output_dir, modality, f"{split_name}.csv")
        df = pd.DataFrame(csv_data[split_name], columns=["image_path", "label"])
        df.to_csv(csv_path, index=False)

print("✅ iris and fingerprint modalities processed and CSVs generated successfully!")


Processing fingerprint: 100%|████████████████████████████████████████████████████████| 100/100 [01:00<00:00,  1.65it/s]

✅ iris and fingerprint modalities processed and CSVs generated successfully!
